In [72]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score

from catboost import CatBoostRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/Blue Book for Bulldozers/TrainAndValid.csv", 
                 low_memory=False,
                parse_dates=["saledate"])

In [3]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for label, content in df.items():
    if content.isna().mean() > 0:
        print(f"{label} is missing: {content.isna().mean():.3f}%. ")

auctioneerID is missing: 0.049%. 
MachineHoursCurrentMeter is missing: 0.643%. 
UsageBand is missing: 0.821%. 
fiSecondaryDesc is missing: 0.341%. 
fiModelSeries is missing: 0.858%. 
fiModelDescriptor is missing: 0.819%. 
ProductSize is missing: 0.525%. 
Drive_System is missing: 0.741%. 
Enclosure is missing: 0.001%. 
Forks is missing: 0.521%. 
Pad_Type is missing: 0.803%. 
Ride_Control is missing: 0.630%. 
Stick is missing: 0.803%. 
Transmission is missing: 0.544%. 
Turbocharged is missing: 0.803%. 
Blade_Extension is missing: 0.937%. 
Blade_Width is missing: 0.937%. 
Enclosure_Type is missing: 0.937%. 
Engine_Horsepower is missing: 0.937%. 
Hydraulics is missing: 0.200%. 
Pushblock is missing: 0.937%. 
Ripper is missing: 0.741%. 
Scarifier is missing: 0.937%. 
Tip_Control is missing: 0.937%. 
Tire_Size is missing: 0.763%. 
Coupler is missing: 0.465%. 
Coupler_System is missing: 0.891%. 
Grouser_Tracks is missing: 0.891%. 
Hydraulics_Flow is missing: 0.891%. 
Track_Type is missing: 0.

In [5]:
df["saledate"].dtype

dtype('<M8[ns]')

In [6]:
def clean_some_data(df):
    df["DayOfSale"] = df["saledate"].dt.day
    df["MonthOfSale"] = df["saledate"].dt.month
    df["YearOfSale"] = df["saledate"].dt.year

    df.drop("saledate", axis=1, inplace=True)
    return df

In [7]:
df = clean_some_data(df)

In [8]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,DayOfSale,MonthOfSale,YearOfSale
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,521D,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,16,11,2006
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,950FII,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,26,3,2004
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,2,2004
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,5,2011
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,7,2009


In [9]:
cat_features = [c for c in df if not pd.api.types.is_numeric_dtype(df[c])]
num_features = [c for c in df if pd.api.types.is_numeric_dtype(df[c]) and c != "SalePrice" and c != "SalesID"]

In [10]:
X = df.drop(["SalesID", "SalePrice"], axis=1)
y = df["SalePrice"]

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)

In [12]:
cat_imputer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

In [62]:
num_imputer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [63]:
preprocessor = ColumnTransformer(transformers=[
    ("num_imputer", num_imputer, num_features),
    ("cat_imputer", cat_imputer, cat_features)
], remainder="passthrough")

In [64]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", CatBoostRegressor(verbose=False, 
                                task_type="GPU",
                               devices='0'))
])

In [65]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MachineID', 'ModelID',
                                                   'datasource', 'auctioneerID',
                                                   'YearMade',
                                                   'MachineHoursCurrentMeter',
                                                   'DayOfSale', 'MonthOfSale',
                                                   'YearOfSale']),
                                                 ('cat_imputer',
                                                  Pipeline(st...
                                                   'state', 'ProductGroup',
                                                   'ProductGroupDesc',
                                                   'Drive_System', 'Enclosure',
                                                   'Forks', 'Pad_Type',
                                                   'Ride_Control', 'Stick',
                                                   'Transmission',
                                                   'Turbocharged',
                                                   'Blade_Extension',
                                                   'Blade_Width',
                                                   'Enclosure_Type',
                                                   'Engine_Horsepower',
                                                   'Hydraulics', 'Pushblock',
                                                   'Ripper', 'Scarifier',
                                                   'Tip_Control', 'Tire_Size',
                                                   'Coupler', ...])])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x0000029EBE21D970>)])

In [66]:
model.score(X_valid, y_valid)

0.87772163897861

In [67]:
y_preds = model.predict(X_valid)

In [73]:
r2_score(y_valid, y_preds)

0.87772163897861

In [18]:
params = {
    "preprocessor__num_imputer__imputer__strategy": ["mean", "median", "most_frequent"],
    "preprocessor__cat_imputer__imputer__strategy": ["constant", "most_frequent"],
    "model__task_type": ["GPU"],
    "model__devices": ["0"]
}

In [19]:
gs_model = GridSearchCV(model, params, cv=5)

In [20]:
gs_model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num_imputer',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['MachineID',
                                                                          'ModelID',
                                                                          'datasource',
                                                                          'auctioneerID',
                                                                          'YearMade',
                                                                          'MachineHoursCurrentMeter',
                                                                          'DayOfSale',
                                                                          'MonthOfSale',
                                                                          'YearOfSale']...
                                                                          'Ripper',
                                                                          'Scarifier',
                                                                          'Tip_Control',
                                                                          'Tire_Size',
                                                                          'Coupler', ...])])),
                                       ('model',
                                        <catboost.core.CatBoostRegressor object at 0x0000029E96E494C0>)]),
             param_grid={'model__devices': ['0'], 'model__task_type': ['GPU'],
                         'preprocessor__cat_imputer__imputer__strategy': ['constant',
                                                                          'most_frequent'],
                         'preprocessor__num_imputer__imputer__strategy': ['mean',
                                                                          'median',
                                                                          'most_frequent']})

In [21]:
gs_model.score(X_valid, y_valid)

0.8774804040972592